In [4]:
!pip install -q transformers accelerate bitsandbytes
!pip install -q huggingface_hub

In [ ]:
from huggingface_hub import login
# login(token="")


In [11]:
dataset_path = "/workspace/강원_travel_qa_dataset.jsonl"

In [12]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="강원_travel_qa_dataset.jsonl")

dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 22302
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 2478
    })
})


In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "Bllossom/llama-3.2-Korean-Bllossom-AICA-5B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_8bit=False
)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"], 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [23]:
def format_example(example):
    instruction = example["instruction"]
    input_text = example["input"]
    output_text = example["output"]

    if input_text:
        prompt = f"### 질문: {instruction}\n{input_text}\n\n### 답변: {output_text}"
    else:
        prompt = f"### 질문: {instruction}\n\n### 답변: {output_text}"
    return {"text": prompt}

dataset = dataset.map(format_example)

Map:   0%|          | 0/22302 [00:00<?, ? examples/s]

Map:   0%|          | 0/2478 [00:00<?, ? examples/s]

In [24]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text", "instruction", "input", "output"])

Map:   0%|          | 0/22302 [00:00<?, ? examples/s]

Map:   0%|          | 0/2478 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    save_strategy="no",           
    eval_strategy="no",     
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,             
    warmup_steps=50,
    fp16=True,
    push_to_hub=True,
    hub_model_id="CHOROROK/llama3.2-travel-finetuned",  # 각자 허깅페이스 username으로 변경!!!
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

In [ ]:
trainer.train()
trainer.model.push_to_hub("CHOROROK/llama3.2-travel-finetuned") # 각자 username으로 변경해야해요!!!
tokenizer.push_to_hub("CHOROROK/llama3.2-travel-finetuned") #이거도!!!

Step,Training Loss,Validation Loss
100,1.662600,1.295348


KeyboardInterrupt: 

테스트

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_model = "Bllossom/llama-3.2-Korean-Bllossom-AICA-5B"
ft_model = "CHOROROK/llama3.2-travel-finetuned"

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    torch_dtype="auto"
)

model = PeftModel.from_pretrained(model, ft_model)

In [ ]:
question = "Q: 강릉에 부모님이랑 갈 만한곳 추천해줘\nA:"

result = pipe(
    question,
    max_new_tokens=200,         
    temperature=0.7,            
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.2,
    no_repeat_ngram_size=3
)

print(result[0]["generated_text"])

In [ ]:
prompt = """너는 따뜻하고 친절한 여행 가이드야. 
친절한 말투로 대답해줘. 
여행지를 설명할 때는 분위기와 함께 추천 이유를 구체적으로 말하고 문장 끝맺음을 제대로 해줘. 
절대로 말줄임표(…)를 사용하지 마. 

Q: 강릉에 부모님이랑 갈 만한곳 추천해줘
A:"""

result = pipe(
    prompt,
    max_new_tokens=200,         
    temperature=0.7,            
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.2,
    no_repeat_ngram_size=3
)
print(result[0]["generated_text"])
